In [6]:
from collections import namedtuple
import os
os.getcwd()
os.chdir("../")
os.getcwd()

'c:\\Projects\\CODEBASE\\DeepCNN_Classifier'

In [7]:
from dataclasses import dataclass
from pathlib import Path
from typing import List

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_include_top: bool
    params_learning_rate: float
    params_classes: int
    params_weights:str

In [8]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml,create_directories

In [9]:
class ConfigurationManager:
    def __init__(self,
            config_filepath= CONFIG_FILE_PATH,
            parms_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.parms = read_yaml(parms_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self)->PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        create_directories([config.root_dir])
        prepare_base_model_config = PrepareBaseModelConfig(
                root_dir=Path(config.root_dir),
                base_model_path= Path(config.base_model_path),
                updated_base_model_path= Path(config.updated_base_model_path),
                params_image_size=self.parms.IMAGE_SIZE,
                params_include_top= self.parms.INCLUDE_TOP,
                params_learning_rate= self.parms.LEARNING_RATE,
                params_classes= self.parms.CLASSES,
                params_weights= self.parms.WEIGHTS,
                )
        return prepare_base_model_config       

In [13]:
import tensorflow as tf
# creating component
class PrepareBaseModel:
    def __init__(self,config: PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )
        
        self.save_model(path=self.config.base_model_path,model=self.model)

    @staticmethod
    def  _prepare_full_model(model,classes,freeze_all,freeze_till,learning_rate):
        if  freeze_all:
            model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                layer.trainable=False
        
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
                                units=classes,
                                activation="softmax",
        )(flatten_in)

        full_model = tf.keras.models.Model(
                    inputs=model.input,
                    outputs=prediction
        )

        full_model.compile(
            optimizer= tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss = tf.keras.losses.CategoricalCrossentropy(),
            metrics= ['accuracy']
            )

        full_model.summary()
        return full_model


    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )
        self.save_model(path = self.config.updated_base_model_path,model= self.full_model)

    @staticmethod    
    def save_model(path:Path,model:tf.keras.Model):
        model.save(path)


In [14]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config =prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise(e)




Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [5]:
x = [5,4,7,8,9]
x[:-2]

[5, 4, 7]